# Cleaned data to curated data (filter and transform using business logic)

In [2]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC") # fix timestamps loaded by spark
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/07/30 15:44:03 WARN Utils: Your hostname, DESKTOP-SATV84A resolves to a loopback address: 127.0.1.1; using 172.26.254.29 instead (on interface eth0)
23/07/30 15:44:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/30 15:44:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/30 15:44:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Yellow Taxis

In [3]:
# make directories for curated data

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/curated/'):
    os.makedirs('../data/curated/')
    
if not os.path.exists('../data/curated/tlc'):
    os.makedirs('../data/curated/tlc')
    
if not os.path.exists('../data/curated/tlc/yellow'):
    os.makedirs('../data/curated/tlc/yellow')
    
if not os.path.exists('../data/curated/tlc/yellow/2022'):
    os.makedirs('../data/curated/tlc/yellow/2022')
    
if not os.path.exists('../data/curated/tlc/yellow/2023'):
    os.makedirs('../data/curated/tlc/yellow/2023')

### Load in data

In [4]:
yellow_cleaned_dir = '../data/cleaned/tlc/yellow/'

yellow_curated_dir = '../data/curated/tlc/yellow/'

sdf_yellow_2022_01 = spark.read.parquet(f"{yellow_cleaned_dir}2022/2022-01.parquet")

sdf_yellow_2022_all = spark.read.parquet(f"{yellow_cleaned_dir}2022/*")

sdf_yellow_2023_01 = spark.read.parquet(f"{yellow_cleaned_dir}2023/2023-01.parquet")

sdf_yellow_2023_02 = spark.read.parquet(f"{yellow_cleaned_dir}2023/2023-02.parquet")

#sdf_yellow_2022_all = spark.read.parquet('../data/cleaned/yellow/2022')



In [52]:
sdf_yellow_2022_01.schema

StructType([StructField('vendorid', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('ratecodeid', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('pulocationid', IntegerType(), True), StructField('dolocationid', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True)])

In [58]:
#sdf_yellow_2022_01.show(5)

sdf_yellow_2022_01.where(
    (F.col('payment_type') == 2)
    & (F.col('tip_amount') > 0)
    & (F.col('tip_amount') > 0)
    ).limit(5)

vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
2,2022-01-01 02:39:26,2022-01-01 02:45:53,1,1.73,1,N,137,229,2,-7.5,-0.5,-0.5,3.0,0.0,-0.3,-8.3,-2.5,0.0
2,2022-01-01 05:37:52,2022-01-01 06:10:00,2,8.37,1,N,90,7,2,-27.5,-0.5,-0.5,9.39,0.0,-0.3,-21.91,-2.5,0.0
2,2022-01-01 16:40:12,2022-01-01 17:16:57,2,18.17,2,N,132,48,2,-52.0,0.0,-0.5,12.37,-6.55,-0.3,-50.73,-2.5,-1.25
2,2022-01-01 17:53:58,2022-01-01 18:14:15,1,2.3,5,N,164,142,2,13.3,0.0,0.5,2.66,0.0,0.3,19.26,2.5,0.0
2,2022-01-01 17:19:45,2022-01-01 17:33:03,1,2.25,1,N,170,236,2,-11.5,0.0,-0.5,2.96,0.0,-0.3,-11.84,-2.5,0.0


In [81]:
# Test filters here
#sdf_yellow_2022_01.schema

payment_cols = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee']

print("Total trips: ")

print(sdf_yellow_2022_01.count())

print("Credit card payment trips with valid total: ")

print(sdf_yellow_2022_01.where((F.col('payment_type') == 1) & (F.col('total_amount') == sum(sdf_yellow_2022_01[col] for col in payment_cols))).count())

print("Trips with valid total: ")

print(sdf_yellow_2022_01.where(F.col('total_amount') == sum(sdf_yellow_2022_01[col] for col in payment_cols)).count())

print("Cash payment trips with valid total: ")

print(sdf_yellow_2022_01.where((F.col('payment_type') == 2) & (F.col('total_amount') == sum(sdf_yellow_2022_01[col] for col in payment_cols))).count())

sdf_yellow_2022_01.where(
    (F.col('payment_type') == 1) 
    & (F.col('total_amount') == sum(sdf_yellow_2022_01[col] for col in payment_cols))
    & (F.col('tolls_amount') > 0)
    & (F.col('airport_fee') > 0)
    ).show(5)

#sdf_yellow_2022_01.filter(F.col('passenger_count') == 5).limit(5)

Total trips: 
2463931
Credit card payment trips with valid total: 
1039104
Trips with valid total: 
1399649
Cash payment trips with valid total: 
348632


vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
2,2022-01-01 18:04:06,2022-01-01 18:40:52,1,9.7,1,N,138,48,1,34.5,0.5,0.5,9.22,6.55,0.3,55.32,2.5,1.25
2,2022-01-01 00:17:55,2022-01-01 00:27:45,1,4.22,1,N,138,92,2,14.0,0.5,0.5,0.0,0.0,0.3,16.55,0.0,1.25
2,2022-01-01 00:55:48,2022-01-01 01:14:24,1,6.67,1,N,138,229,1,21.0,0.5,0.5,15.0,0.0,0.3,41.05,2.5,1.25
1,2022-01-01 00:10:27,2022-01-01 00:41:59,1,17.1,2,N,132,170,1,52.0,3.75,0.5,15.75,6.55,0.3,78.85,2.5,1.25
2,2022-01-01 00:50:27,2022-01-01 00:59:28,1,2.66,1,N,132,10,2,10.0,0.5,0.5,0.0,0.0,0.3,12.55,0.0,1.25
2,2022-01-01 00:31:06,2022-01-01 00:58:26,3,19.14,2,N,132,263,1,52.0,0.0,0.5,12.37,6.55,0.3,75.47,2.5,1.25
2,2022-01-01 00:42:45,2022-01-01 00:56:15,2,6.49,1,N,138,112,1,19.5,0.5,0.5,6.62,0.0,0.3,28.67,0.0,1.25
2,2022-01-01 00:56:26,2022-01-01 01:25:09,1,18.81,2,N,132,148,1,52.0,0.0,0.5,11.31,0.0,0.3,67.86,2.5,1.25
1,2022-01-01 00:33:42,2022-01-01 01:01:49,1,19.5,1,N,132,47,1,52.0,1.75,0.5,0.0,6.55,0.3,61.1,0.0,1.25
1,2022-01-01 00:34:34,2022-01-01 01:08:45,2,20.7,2,N,132,236,1,52.0,3.75,0.5,11.3,0.0,0.3,67.85,2.5,1.25


In [90]:
sdf_yellow_2022_01.where(
    (F.col('airport_fee') == 0)
    & (F.col('pulocationid') != 132)
    & (F.col('pulocationid') != 138)
    | (F.col('pulocationid') == 132)
    & (F.col('airport_fee') == 1.25)
    | (F.col('pulocationid') == 138)
    & (F.col('airport_fee') == 1.25)
    ).where(
        (F.col('airport_fee') != 1.25)
        & (F.col('pulocationid') == 132)
        & (F.col('pulocationid') == 138)
        ).show(5)

'''
sdf_yellow_2022_01.withColumn(
    'is_valid_record',
    # when we have a positive distance/passenger/total amount then True
    # else False
    F.when(
        (F.col('airport_fee') == 0)
        & (F.col('pulocationid') != 132)
        & (F.col('pulocationid') != 138)
        | (F.col('pulocationid') == 132)
        & (F.col('airport_fee') == 1.25)
        | (F.col('pulocationid') == 138)
        & (F.col('airport_fee') == 1.25),
        True
    ).otherwise(False)
).where(F.col('airport_fee') == 1.25)
'''
sdf_yellow_2022_01.where(
    (F.col('vendorid') == 1)
    | (F.col('vendorid') == 2)
    ).where(
        (F.col('vendorid') == 1)
        & (F.col('vendorid') != 2)
    )


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|vendorid|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|ratecodeid|store_and_fwd_flag|pulocationid|dolocationid|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+---------

vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-01-01 00:35:40,2022-01-01 00:53:29,2,3.8,1,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1,2.1,1,N,236,42,1,8.0,0.5,0.5,4.0,0.0,0.3,13.3,0.0,0.0
1,2022-01-01 00:40:15,2022-01-01 01:09:48,1,10.3,1,N,138,161,1,33.0,3.0,0.5,13.0,6.55,0.3,56.35,2.5,0.0
1,2022-01-01 00:33:52,2022-01-01 00:47:28,3,4.2,1,N,148,141,1,14.0,2.5,0.5,3.45,0.0,0.3,20.75,2.5,0.0
1,2022-01-01 00:53:54,2022-01-01 01:05:20,2,2.2,1,N,237,107,1,9.5,2.5,0.5,2.55,0.0,0.3,15.35,2.5,0.0
1,2022-01-01 00:00:44,2022-01-01 00:05:29,1,0.2,1,N,7,7,2,5.0,0.5,0.5,0.0,0.0,0.3,6.3,0.0,0.0
1,2022-01-01 00:35:50,2022-01-01 00:48:33,2,3.9,1,N,107,263,1,13.0,3.0,0.5,3.35,0.0,0.3,20.15,2.5,0.0
1,2022-01-01 00:49:14,2022-01-01 00:58:29,2,3.2,1,N,263,107,1,11.0,3.0,0.5,2.95,0.0,0.3,17.75,2.5,0.0
1,2022-01-01 00:06:10,2022-01-01 00:08:58,1,0.0,1,N,161,161,4,2.5,3.0,0.5,0.0,0.0,0.3,6.3,2.5,0.0
1,2022-01-01 00:09:01,2022-01-01 00:17:15,1,1.2,1,N,161,43,1,7.0,3.0,0.5,2.15,0.0,0.3,12.95,2.5,0.0


### Use business logic to filter and transform data

In [61]:
payment_cols = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee']

from pyspark.sql import functions as F

# 2022
for month in range(1,13):

    year = 2022
    sdf_cleaned = spark.read.parquet(f"{yellow_cleaned_dir}2022/2022-{str(month).zfill(2)}.parquet")  

    #laguardia_id = 
    #jfk_id = 
    
    '''
    sdf = sdf.withColumn(
        'is_valid_record',
        # when we have a positive distance/passenger/total amount then True
        # else False
        F.when(
            (F.col('trip_distance') > 0)
            & (F.col('passenger_count') > 0)
            & (F.col('total_amount') > 0),
            True
        ).otherwise(False)
    )
    '''
    
    # may have to go through data and apply invalid record boolean attribute first for more complex business logic checking such as airport fee matching pickup at laguardia / jfk
    
    # remove invalid records by applying business logic
    sdf_cleaned = \
    sdf_cleaned.where(
        (F.year(F.col('tpep_pickup_datetime')) == year) # ensure that this trip was initiated within the correct year (assuming a trip belongs to a year based on pickup time)
        (F.month(F.col('tpep_pickup_datetime')) == month)  # ensure that this trip was initiated within the correct month (assuming a trip belongs to a month based on pickup time)
        & (F.unix_timestamp('tpep_pickup_datetime') < F.unix_timestamp('tpep_dropoff_datetime')) # make sure pickup time is earlier than dropoff time
        & (F.col('passenger_count') > 0) # ensure non-zero passenger count, trip distance and total amount
        & (F.col('trip_distance') > 0)
        & (F.col('total_amount') > 0) 
        & (F.col('fare_amount') >= 0) # ensure non-negative fees
        & (F.col('extra') >= 0) 
        & (F.col('mta_tax') >= 0) 
        & (F.col('tip_amount') >= 0) 
        & (F.col('tolls_amount') >= 0)
        & (F.col('improvement_surcharge') >= 0) 
        & (F.col('congestion_surcharge') >= 0)  
        & (F.col('payment_type') == 1) # include only credit card payments as cash tips are not included in this data
        & (F.col('total_amount') == sum(sdf_yellow_2022_01[col] for col in payment_cols)) # check the total amount equals the sum of all other fees
    ).where(
        (F.col('vendorid') == 1)
        | (F.col('vendorid') == 2)  
    ).where(
        (F.col('airport_fee') == 0)
        & (F.col('pulocationid') != 132)
        & (F.col('pulocationid') != 138)
        | (F.col('pulocationid') == 132)
        & (F.col('airport_fee') == 1.25)
        | (F.col('pulocationid') == 138)
        & (F.col('airport_fee') == 1.25)  
    ).withColumn(
        'store_and_fwd_flag',
        (F.col("store_and_fwd_flag") == 'Y').cast('BOOLEAN')
    )
    
    '''
    sdf_cleaned = sdf_cleaned.withColumn(
        'store_and_fwd_flag',
        (F.col("store_and_fwd_flag") == 'Y').cast('BOOLEAN')
    )
    '''
    
    
    
    
    
        
    sdf_cleaned \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{yellow_curated_dir}2022/2022-{str(month).zfill(2)}.parquet")

# 2023
for month in range(1,5):

    year = 2023
    sdf_cleaned = spark.read.parquet(f"{yellow_cleaned_dir}2023/2023-{str(month).zfill(2)}.parquet")

    # select all columns from the existing cleaned dataframe and cast it to the required schema
    sdf_cleaned = sdf_cleaned \
        .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_cleaned.columns)])
     
    # apply business logic
    
    sdf_cleaned = sdf_cleaned.withColumn(
        'store_and_fwd_flag',
        (F.col("store_and_fwd_flag") == 'Y').cast('BOOLEAN')
    )
    
    
    
    
    
       
    sdf_cleaned \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{yellow_curated_dir}2023/2023-{str(month).zfill(2)}.parquet")
    
'''
# renaming
sdf.withColumnRenamed(
    'column_from',
    'column_to'
)

# example 1 for converting data types
sdf.withColumn(
    'column_to',
    F.col('column_from').cast('data type')
)
'''

TypeError: 'Column' object is not callable

In [ ]:
sdf.show(1, vertical=True, truncate=100)

In [ ]:
# double check curated 2022 data

sdf1 = spark.read.schema(sdf_schema).parquet('../data/cleaned/tlc/yellow/2022/*')

sdf1.show(1, vertical=True, truncate=100)

''' checking raw vs cleaned dataset shape and raw dataset shape of jan 2022
sdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')

sdf2.show(1, vertical=True, truncate=100)

print(sdf1.count())

print(sdf2.count())

sdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf.show(1, vertical=True, truncate=100)

print(sdf.count())
'''

In [ ]:
# double check curated 2023 data

sdf = spark.read.schema(sdf_schema).parquet('../data/cleaned/tlc/yellow/2023/*')

sdf.show(1, vertical=True, truncate=100)

## Green Taxis

In [62]:
# make directories for cleaned data

import os

if not os.path.exists('../data/cleaned/'):
    os.makedirs('../data/cleaned/')
    
if not os.path.exists('../data/cleaned/tlc'):
    os.makedirs('../data/cleaned/tlc')
    
if not os.path.exists('../data/cleaned/tlc/green'):
    os.makedirs('../data/cleaned/tlc/green')
    
if not os.path.exists('../data/cleaned/tlc/green/2022'):
    os.makedirs('../data/cleaned/tlc/green/2022')
    
if not os.path.exists('../data/cleaned/tlc/green/2023'):
    os.makedirs('../data/cleaned/tlc/green/2023')

### Check data types

In [63]:
green_raw_dir = '../data/raw/tlc/green/'

sdf_green_2022_01 = spark.read.parquet(f"{green_raw_dir}2022/2022-01.parquet")

sdf_green_2022_all = spark.read.parquet(f"{green_raw_dir}2022")

sdf_green_2023_01 = spark.read.parquet(f"{green_raw_dir}2023/2023-01.parquet")

sdf_green_2023_02 = spark.read.parquet(f"{green_raw_dir}2023/2023-02.parquet")

#sdf_green_2022_all = spark.read.parquet('../data/raw/green/2022')



In [46]:
sdf_green_2022_01.schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', DoubleType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', IntegerType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', DoubleType(), True), StructField('trip_type', DoubleType(), True), StructField('congestion_surcharge', Doubl

In [47]:
sdf_green_2022_all.schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', DoubleType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', IntegerType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', DoubleType(), True), StructField('trip_type', DoubleType(), True), StructField('congestion_surcharge', Doubl

In [48]:
sdf_green_2023_01.schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', DoubleType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', IntegerType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', DoubleType(), True), StructField('trip_type', DoubleType(), True), StructField('congestion_surcharge', Doubl

In [49]:
sdf_green_2023_02.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', LongType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge', Doubl

### Schemas appear to be incorrect for all of 2022, so we will adjust all datasets to line up with the schema of Feb 2023 data and ensure consistent lowercase casing

In [64]:
from pyspark.sql import functions as F

'''
# renaming
sdf.withColumnRenamed(
    'column_from',
    'column_to'
)

# example 1 for converting data types
sdf.withColumn(
    'column_to',
    F.col('column_from').cast('data type')
)
'''

# RatecodeID and passenger_count also need to be converted to int

sdf_green_2023_02 = sdf_green_2023_02.withColumn(
    'ratecodeid', 
    F.col('RatecodeID').cast('Integer') 
)

sdf_green_2023_02 = sdf_green_2023_02.withColumn(
    'passenger_count',
    F.col('passenger_count').cast('Integer')
)


sdf_green_2023_02.schema



StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('ratecodeid', IntegerType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge',

In [65]:
# now, we want to ensure everything has consistent casing to make our lives easier
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_green_2023_02.columns]
sdf_green_2023_02 = sdf_green_2023_02.select(*consistent_col_casing)

# this will be used in the cell below when reading in
sdf_schema = sdf_green_2023_02.schema
sdf_schema

StructType([StructField('vendorid', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('ratecodeid', IntegerType(), True), StructField('pulocationid', IntegerType(), True), StructField('dolocationid', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge',

In [66]:
# convert all raw datasets to required schema

green_raw_dir = '../data/raw/tlc/green/'
    
green_clean_dir = '../data/cleaned/tlc/green/'

# 2022
for month in range(1,13):

    month = str(month).zfill(2) 

    sdf_malformed = spark.read.parquet(f"{green_raw_dir}2022/2022-{month}.parquet")

    # select all columns from the existing malformed dataframe and cast it to the required schema
    sdf_malformed = sdf_malformed \
        .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)])
        
    sdf_malformed \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{green_clean_dir}2022/2022-{month}.parquet")

# 2023
for month in range(1,5):
    month = str(month).zfill(2) 

    sdf_malformed = spark.read.parquet(f"{green_raw_dir}2023/2023-{month}.parquet")

    # select all columns from the existing malformed dataframe and cast it to the required schema
    sdf_malformed = sdf_malformed \
        .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)])
        
    sdf_malformed \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{green_clean_dir}2023/2023-{month}.parquet")


In [67]:
# double check schema of 2022 cleaned data

sdf1 = spark.read.schema(sdf_schema).parquet('../data/cleaned/tlc/green/2022/*')

sdf1.show(1, vertical=True, truncate=100)

''' checking raw vs cleaned dataset shape and raw dataset shape of jan 2022
sdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')

sdf2.show(1, vertical=True, truncate=100)

print(sdf1.count())

print(sdf2.count())

sdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf.show(1, vertical=True, truncate=100)

print(sdf.count())
'''

-RECORD 0------------------------------------
 vendorid              | 1                   
 lpep_pickup_datetime  | 2022-03-01 00:24:14 
 lpep_dropoff_datetime | 2022-03-01 00:34:04 
 store_and_fwd_flag    | N                   
 ratecodeid            | 1                   
 pulocationid          | 74                  
 dolocationid          | 151                 
 passenger_count       | 1                   
 trip_distance         | 2.3                 
 fare_amount           | 9.5                 
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 ehail_fee             | null                
 improvement_surcharge | 0.3                 
 total_amount          | 10.8                
 payment_type          | 2                   
 trip_type             | 1                   
 congestion_surcharge  | 0.0                 
only showing top 1 row



" checking raw vs cleaned dataset shape and raw dataset shape of jan 2022\nsdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')\n\nsdf2.show(1, vertical=True, truncate=100)\n\nprint(sdf1.count())\n\nprint(sdf2.count())\n\nsdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')\n\nsdf.show(1, vertical=True, truncate=100)\n\nprint(sdf.count())\n"

In [68]:
# double check schema of 2023 cleaned data

sdf = spark.read.schema(sdf_schema).parquet('../data/cleaned/tlc/green/2023/*')

sdf.show(1, vertical=True, truncate=100)

-RECORD 0------------------------------------
 vendorid              | 2                   
 lpep_pickup_datetime  | 2023-03-01 00:25:10 
 lpep_dropoff_datetime | 2023-03-01 00:35:47 
 store_and_fwd_flag    | N                   
 ratecodeid            | 1                   
 pulocationid          | 82                  
 dolocationid          | 196                 
 passenger_count       | 1                   
 trip_distance         | 2.36                
 fare_amount           | 13.5                
 extra                 | 1.0                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 ehail_fee             | null                
 improvement_surcharge | 1.0                 
 total_amount          | 16.0                
 payment_type          | 2                   
 trip_type             | 1                   
 congestion_surcharge  | 0.0                 
only showing top 1 row



## High Volume For-Hire Vehicles

In [56]:
# make directories for cleaned data

import os

if not os.path.exists('../data/cleaned/'):
    os.makedirs('../data/cleaned/')
    
if not os.path.exists('../data/cleaned/tlc'):
    os.makedirs('../data/cleaned/tlc')
    
if not os.path.exists('../data/cleaned/tlc/fhvhv'):
    os.makedirs('../data/cleaned/tlc/fhvhv')
    
if not os.path.exists('../data/cleaned/tlc/fhvhv/2022'):
    os.makedirs('../data/cleaned/tlc/fhvhv/2022')
    
if not os.path.exists('../data/cleaned/tlc/fhvhv/2023'):
    os.makedirs('../data/cleaned/tlc/fhvhv/2023')

### Check data types

In [57]:
fhvhv_raw_dir = '../data/raw/tlc/fhvhv/'

sdf_fhvhv_2022_01 = spark.read.parquet(f"{fhvhv_raw_dir}2022/2022-01.parquet")

sdf_fhvhv_2022_all = spark.read.parquet(f"{fhvhv_raw_dir}2022")

sdf_fhvhv_2023_01 = spark.read.parquet(f"{fhvhv_raw_dir}2023/2023-01.parquet")

sdf_fhvhv_2023_02 = spark.read.parquet(f"{fhvhv_raw_dir}2023/2023-02.parquet")

#sdf_fhvhv_2022_all = spark.read.parquet(f"{fhvhv_raw_dir}2022")



In [58]:
sdf_fhvhv_2022_01.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_re

In [59]:
sdf_fhvhv_2022_all.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_re

In [60]:
sdf_fhvhv_2023_01.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_re

In [61]:
sdf_fhvhv_2023_02.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('sha

### Schemas appear to be incorrect for all of 2022, so we will adjust all datasets to line up with the schema of Feb 2023 data and ensure consistent lowercase casing

In [69]:
from pyspark.sql import functions as F

# RatecodeID and passenger_count also need to be converted to int
'''
# renaming
sdf.withColumnRenamed(
    'column_from',
    'column_to'
)

# example 1 for converting data types
sdf.withColumn(
    'column_to',
    F.col('column_from').cast('data type')
)
'''

# trip_time needs to be converted to integer

sdf_fhvhv_2023_02 = sdf_fhvhv_2023_02.withColumn(
    'trip_time', 
    F.col('trip_time').cast('Integer') 
)

sdf_fhvhv_2023_02.schema



StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', IntegerType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('

In [70]:
# now, we want to ensure everything has consistent casing to make our lives easier
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_fhvhv_2023_02.columns]
sdf_fhvhv_2023_02 = sdf_fhvhv_2023_02.select(*consistent_col_casing)

# this will be used in the cell below when reading in
sdf_schema = sdf_fhvhv_2023_02.schema
sdf_schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('pulocationid', IntegerType(), True), StructField('dolocationid', IntegerType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', IntegerType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('

In [71]:
# convert all raw datasets to required schema

fhvhv_raw_dir = '../data/raw/tlc/fhvhv/'
    
fhvhv_clean_dir = '../data/cleaned/tlc/fhvhv/'

# 2022
for month in range(1,13):

    month = str(month).zfill(2) 

    sdf_malformed = spark.read.parquet(f"{fhvhv_raw_dir}2022/2022-{month}.parquet")

    # select all columns from the existing malformed dataframe and cast it to the required schema
    sdf_malformed = sdf_malformed \
        .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)])
        
    sdf_malformed \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{fhvhv_clean_dir}2022/2022-{month}.parquet")

# 2023
for month in range(1,5):
    month = str(month).zfill(2) 

    sdf_malformed = spark.read.parquet(f"{fhvhv_raw_dir}2023/2023-{month}.parquet")

    # select all columns from the existing malformed dataframe and cast it to the required schema
    sdf_malformed = sdf_malformed \
        .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)])
        
    sdf_malformed \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{fhvhv_clean_dir}2023/2023-{month}.parquet")


In [72]:
# double check schema of 2022 cleaned data

sdf1 = spark.read.schema(sdf_schema).parquet('../data/cleaned/tlc/fhvhv/2022/*')

sdf1.show(1, vertical=True, truncate=100)

''' checking raw vs cleaned dataset shape and raw dataset shape of jan 2022
sdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')

sdf2.show(1, vertical=True, truncate=100)

print(sdf1.count())

print(sdf2.count())

sdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf.show(1, vertical=True, truncate=100)

print(sdf.count())
'''

-RECORD 0-----------------------------------
 hvfhs_license_num    | HV0003              
 dispatching_base_num | B03404              
 originating_base_num | B03404              
 request_datetime     | 2022-01-01 00:05:31 
 on_scene_datetime    | 2022-01-01 00:05:40 
 pickup_datetime      | 2022-01-01 00:07:24 
 dropoff_datetime     | 2022-01-01 00:18:28 
 pulocationid         | 170                 
 dolocationid         | 161                 
 trip_miles           | 1.18                
 trip_time            | 664                 
 base_passenger_fare  | 24.9                
 tolls                | 0.0                 
 bcf                  | 0.75                
 sales_tax            | 2.21                
 congestion_surcharge | 2.75                
 airport_fee          | 0.0                 
 tips                 | 0.0                 
 driver_pay           | 23.03               
 shared_request_flag  | N                   
 shared_match_flag    | N                   
 access_a_

" checking raw vs cleaned dataset shape and raw dataset shape of jan 2022\nsdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')\n\nsdf2.show(1, vertical=True, truncate=100)\n\nprint(sdf1.count())\n\nprint(sdf2.count())\n\nsdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')\n\nsdf.show(1, vertical=True, truncate=100)\n\nprint(sdf.count())\n"

In [73]:
# double check schema of 2023 cleaned data

sdf = spark.read.schema(sdf_schema).parquet('../data/cleaned/tlc/fhvhv/2023/*')

sdf.show(1, vertical=True, truncate=100)

-RECORD 0-----------------------------------
 hvfhs_license_num    | HV0003              
 dispatching_base_num | B03404              
 originating_base_num | B03404              
 request_datetime     | 2023-01-01 00:18:06 
 on_scene_datetime    | 2023-01-01 00:19:24 
 pickup_datetime      | 2023-01-01 00:19:38 
 dropoff_datetime     | 2023-01-01 00:48:07 
 pulocationid         | 48                  
 dolocationid         | 68                  
 trip_miles           | 0.94                
 trip_time            | 1709                
 base_passenger_fare  | 25.95               
 tolls                | 0.0                 
 bcf                  | 0.78                
 sales_tax            | 2.3                 
 congestion_surcharge | 2.75                
 airport_fee          | 0.0                 
 tips                 | 5.22                
 driver_pay           | 27.83               
 shared_request_flag  | N                   
 shared_match_flag    | N                   
 access_a_

## Taxi Zones

In [75]:
# make directories for cleaned data

import os

if not os.path.exists('../data/cleaned/'):
    os.makedirs('../data/cleaned/')
    
if not os.path.exists('../data/cleaned/tlc'):
    os.makedirs('../data/cleaned/tlc')
    
if not os.path.exists('../data/cleaned/tlc/taxi_zones'):
    os.makedirs('../data/cleaned/tlc/taxi_zones')

In [5]:
taxi_zone_lookup = spark.read.option("header",True) \
                   .csv("../data/raw/tlc/taxi_zones/taxi+_zone_lookup.csv")

In [10]:
print(taxi_zone_lookup.limit(5))

print(taxi_zone_lookup.schema)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+

StructType([StructField('LocationID', StringType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])


### Just need to convert the column names to lowercase for this data

In [12]:
from pyspark.sql import functions as F

# converting to lower case
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in taxi_zone_lookup.columns]
taxi_zone_lookup = taxi_zone_lookup.select(*consistent_col_casing)

print(taxi_zone_lookup.limit(5))
print(taxi_zone_lookup.schema)

+----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+

StructType([StructField('locationid', StringType(), True), StructField('borough', StringType(), True), StructField('zone', StringType(), True), StructField('service_zone', StringType(), True)])


In [13]:
# save cleaned data as parquet
taxi_zone_lookup \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet('../data/cleaned/tlc/taxi_zones/taxi_zone_lookup.parquet')

In [14]:
# check cleaned data
taxi_zone_lookup = spark.read.parquet('../data/cleaned/tlc/taxi_zones/taxi_zone_lookup.parquet')

taxi_zone_lookup.printSchema


<bound method DataFrame.printSchema of +----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City